# ES → BBF Salesforce Contact Migration

This notebook migrates Contact records from ES Salesforce to BBF Salesforce.

## Prerequisites
- **Account migration must be completed first**
- ES Accounts with `BBF_New_Id__c` populated = parent Account already exists in BBF

## Process Overview
1. Connect to both ES (source) and BBF (target) Salesforce orgs
2. Query Contacts from ES where parent Account has `BBF_New_Id__c` populated
3. Transform ES Contacts for BBF schema:
   - Map `AccountId` → ES Account's `BBF_New_Id__c` (BBF Account ID)
   - Map standard Contact fields
   - Add `ES_Legacy_ID__c` = ES Contact.Id (for tracking)
4. Insert Contacts to BBF Salesforce
5. Update ES Contacts with `BBF_New_Id__c` = BBF Contact.Id
6. Create ID mapping: ES Contact ID → BBF Contact ID
7. Output results to Excel with color-coded status

## Field Tracking Strategy
**In BBF:** `ES_Legacy_ID__c` stores original ES Contact ID
- Text(18), External ID, Unique

**In ES:** `BBF_New_Id__c` stores new BBF Contact ID
- Text(18)

## Safety
- `TEST_MODE = True` by default (limits to 10 Contacts)
- Skips Contacts where parent Account not yet migrated
- Skips Contacts already migrated (`BBF_New_Id__c` populated)

In [1]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
import os

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("✅ Set-up successful")

Python: C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Pandas: 2.2.3
✅ Set-up successful


In [2]:
# === CONFIGURATION ===

# ES UAT Credentials
ES_USERNAME = "sfdcapi@everstream.net.uat"
ES_PASSWORD = "ZXasqw1234!@#$"
ES_TOKEN = "X0ation2CNmK5C0pV94M6vFYS"
ES_DOMAIN = "test"

# # ES (Source) Credentials - Production
# ES_USERNAME = "sfdcapi@everstream.net"
# ES_PASSWORD = "pV4CAxns8DQtJsBq!"
# ES_TOKEN = "r1uoYiusK19RbrflARydi86TA"
# ES_DOMAIN = "login"  # or 'test' for sandbox

# BBF (Target) Credentials
BBF_USERNAME = "vlettau@everstream.net"
BBF_PASSWORD = "MNlkpo0987)(*&"
BBF_TOKEN = "I4xmQLmm03cXl1O9qI2Z3XAAX"
BBF_DOMAIN = "test"  # or 'login' for production

# Migration Options
TEST_MODE = False  # ⚠️ Set to False to migrate ALL contacts
TEST_LIMIT = 10  # Only used when TEST_MODE = True

# 👤 Contact Owner - Set all migrated contacts to this user
OWNER_ID = "005Ea00000ZOGFZIA5"  # Same as Account migration

# Output Configuration
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"es_bbf_contact_migration_{timestamp}.xlsx"

print("📋 Configuration loaded")
print(f"   TEST_MODE: {TEST_MODE}")
print(f"   Owner ID: {OWNER_ID}")
print(f"   Output: {output_file}")
print("\n⚠️  Note: Bulk API automatically handles batching (200 records/batch)")

📋 Configuration loaded
   TEST_MODE: False
   Owner ID: 005Ea00000ZOGFZIA5
   Output: es_bbf_contact_migration_20260123_114209.xlsx

⚠️  Note: Bulk API automatically handles batching (200 records/batch)


In [3]:
# === CONNECT TO SALESFORCE ORGS ===

print("=" * 80)
print("CONNECTING TO SALESFORCE ORGS")
print("=" * 80)

# Connect to ES (source)
print("\n🔌 Connecting to ES (source)...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

# Connect to BBF (target)
print("\n🔌 Connecting to BBF (target)...")
bbf_sf = Salesforce(
    username=BBF_USERNAME,
    password=BBF_PASSWORD,
    security_token=BBF_TOKEN,
    domain=BBF_DOMAIN,
)
print(f"✅ Connected to BBF: {bbf_sf.sf_instance}")

CONNECTING TO SALESFORCE ORGS

🔌 Connecting to ES (source)...
✅ Connected to ES: everstream--uat.sandbox.my.salesforce.com

🔌 Connecting to BBF (target)...
✅ Connected to BBF: bluebirdnetwork--full.sandbox.my.salesforce.com


In [ ]:
# === QUERY ES CONTACTS ===
# Only Contacts where:
# 1. Parent Account has BBF_New_Id__c populated (Account already migrated)
# 2. Contact does NOT have BBF_New_Id__c populated (Contact not yet migrated)

print("\n" + "=" * 80)
print("QUERYING ES CONTACTS")
print("=" * 80)

# Build query - include Account.BBF_New_Id__c in the query
query = """
    SELECT Id, AccountId, Account.BBF_New_Id__c,
           FirstName, LastName, Email, Phone, Title,
           MailingStreet, MailingCity, MailingState, MailingPostalCode, MailingCountry,
           OtherStreet, OtherCity, OtherState, OtherPostalCode, OtherCountry,
           MobilePhone, HomePhone, Fax,
           Department, Description, Birthdate,
           AssistantName, AssistantPhone, LeadSource
    FROM Contact
    WHERE Account.BBF_New_Id__c != null 
      AND Account.BBF_New_Id__c != ''
      AND (BBF_New_Id__c = null OR BBF_New_Id__c = '')
"""

# Add limit for test mode
if TEST_MODE:
    query += f" LIMIT {TEST_LIMIT}"

print(f"Query: {query}")
print("\nExecuting query...")

result = es_sf.query_all(query)
es_contacts_raw = result["records"]

print(f"✅ Found {len(es_contacts_raw)} contacts from ES query")

# =============================================================================
# DUPLICATE PREVENTION: Check BBF for already-migrated records
# If record exists in BBF, get its Id to update ES BBF_New_Id__c
# =============================================================================
print("\n" + "-" * 80)
print("DUPLICATE PREVENTION: Checking BBF for existing ES_Legacy_ID__c...")
print("-" * 80)

bbf_existing_query = """
SELECT Id, ES_Legacy_ID__c 
FROM Contact 
WHERE ES_Legacy_ID__c != null
"""
bbf_existing_result = bbf_sf.query_all(bbf_existing_query)

# Build lookup: ES_Legacy_ID__c -> BBF Id
existing_bbf_lookup = {r['ES_Legacy_ID__c']: r['Id'] for r in bbf_existing_result['records']}

print(f"   Found {len(existing_bbf_lookup)} Contact records already in BBF")

# Separate: records to migrate vs records that need ES BBF_New_Id__c sync
es_contacts = []
es_needs_sync = []  # Records that exist in BBF but ES.BBF_New_Id__c is null

for contact in es_contacts_raw:
    if contact['Id'] in existing_bbf_lookup:
        # Already in BBF - need to sync ES.BBF_New_Id__c
        es_needs_sync.append({
            'es_id': contact['Id'],
            'bbf_id': existing_bbf_lookup[contact['Id']]
        })
    else:
        # Not in BBF - need to migrate
        es_contacts.append(contact)

print(f"   Records to migrate (not in BBF): {len(es_contacts)}")
print(f"   Records to sync (in BBF, ES.BBF_New_Id__c missing): {len(es_needs_sync)}")

# Sync ES.BBF_New_Id__c for records that already exist in BBF
if len(es_needs_sync) > 0:
    print(f"\n📌 Syncing {len(es_needs_sync)} ES Contact BBF_New_Id__c values...")
    
    sync_updates = [{'Id': item['es_id'], 'BBF_New_Id__c': item['bbf_id']} for item in es_needs_sync]
    
    try:
        sync_results = es_sf.bulk.Contact.update(sync_updates)
        sync_success = sum(1 for r in sync_results if r['success'])
        sync_failed = sum(1 for r in sync_results if not r['success'])
        
        print(f"   ✅ Synced: {sync_success}")
        print(f"   ❌ Failed to sync: {sync_failed}")
        
        if sync_failed > 0:
            print("   First 5 sync failures:")
            fail_count = 0
            for i, r in enumerate(sync_results):
                if not r['success'] and fail_count < 5:
                    print(f"     - {sync_updates[i]['Id']}: {r['errors']}")
                    fail_count += 1
    except Exception as e:
        print(f"   ❌ Error syncing: {e}")

print(f"\n✅ {len(es_contacts)} contacts to migrate (after duplicate check)")

if len(es_contacts) > 0:
    sample = es_contacts[0]
    print(f"\nSample Contact:")
    print(f"  ID:         {sample['Id']}")
    print(f"  Name:       {sample.get('FirstName', '')} {sample.get('LastName', '')}")
    print(f"  ES Account: {sample.get('AccountId', 'N/A')}")
    print(
        f"  BBF Account (target): {sample.get('Account', {}).get('BBF_New_Id__c', 'N/A')}"
    )
elif TEST_MODE:
    print("\n⚠️  No unmigrated contacts found in test set")
    print("   Check: Are there ES Accounts with BBF_New_Id__c populated?")
else:
    print("\n✅ All contacts have been migrated (or no eligible contacts found)!")

In [5]:
# === TRANSFORM FOR BBF ===

print("\n" + "=" * 80)
print("TRANSFORMING CONTACTS FOR BBF")
print("=" * 80)

bbf_contacts = []
skipped_no_bbf_account = []

for es_contact in es_contacts:
    # Get the BBF Account ID from the parent Account
    bbf_account_id = None
    if es_contact.get("Account") and es_contact["Account"].get("BBF_New_Id__c"):
        bbf_account_id = es_contact["Account"]["BBF_New_Id__c"]

    # Safety check: Skip if no BBF Account ID (shouldn't happen due to query filter)
    if not bbf_account_id:
        skipped_no_bbf_account.append(
            {
                "es_id": es_contact["Id"],
                "name": f"{es_contact.get('FirstName', '')} {es_contact.get('LastName', '')}",
                "es_account_id": es_contact.get("AccountId"),
            }
        )
        continue

    bbf_contact = {
        # 🔗 CRITICAL: Use BBF Account ID from parent Account's BBF_New_Id__c
        "AccountId": bbf_account_id,
        # Standard Contact fields
        "FirstName": es_contact.get("FirstName"),
        "LastName": es_contact.get("LastName"),
        "Email": es_contact.get("Email"),
        "Phone": es_contact.get("Phone"),
        "Title": es_contact.get("Title"),
        # Mailing Address
        "MailingStreet": es_contact.get("MailingStreet"),
        "MailingCity": es_contact.get("MailingCity"),
        "MailingState": es_contact.get("MailingState"),
        "MailingPostalCode": es_contact.get("MailingPostalCode"),
        "MailingCountry": es_contact.get("MailingCountry"),
        # Other Address
        "OtherStreet": es_contact.get("OtherStreet"),
        "OtherCity": es_contact.get("OtherCity"),
        "OtherState": es_contact.get("OtherState"),
        "OtherPostalCode": es_contact.get("OtherPostalCode"),
        "OtherCountry": es_contact.get("OtherCountry"),
        # Additional phone fields
        "MobilePhone": es_contact.get("MobilePhone"),
        "HomePhone": es_contact.get("HomePhone"),
        "Fax": es_contact.get("Fax"),
        # Other standard fields
        "Department": es_contact.get("Department"),
        "Description": es_contact.get("Description"),
        "Birthdate": es_contact.get("Birthdate"),
        "AssistantName": es_contact.get("AssistantName"),
        "AssistantPhone": es_contact.get("AssistantPhone"),
        "LeadSource": es_contact.get("LeadSource"),
        # 👤 Set owner
        "OwnerId": OWNER_ID,
        # 🔗 Store ES Contact ID for tracking
        "ES_Legacy_ID__c": es_contact["Id"],
    }

    bbf_contacts.append(bbf_contact)

print(f"✅ Transformed {len(bbf_contacts)} contacts")
if len(bbf_contacts) > 0:
    print(
        f"   - Mapped {len([k for k in bbf_contacts[0].keys() if not k.startswith('_')])} fields per contact"
    )
    print(f"   - Set OwnerId to {OWNER_ID}")
    print(f"   - Set ES_Legacy_ID__c for tracking")

if len(skipped_no_bbf_account) > 0:
    print(
        f"\n⚠️  Skipped {len(skipped_no_bbf_account)} contacts (no BBF Account ID found)"
    )
    for skip in skipped_no_bbf_account[:5]:
        print(f"   - {skip['name']} (ES Account: {skip['es_account_id']})")


TRANSFORMING CONTACTS FOR BBF
✅ Transformed 15665 contacts
   - Mapped 27 fields per contact
   - Set OwnerId to 005Ea00000ZOGFZIA5
   - Set ES_Legacy_ID__c for tracking


In [6]:
# === INSERT TO BBF ===

print("\n" + "=" * 80)
print("INSERTING CONTACTS TO BBF")
print("=" * 80)

if len(bbf_contacts) == 0:
    print("⚠️  No contacts to insert")
    successful_inserts = []
    failed_inserts = []
else:
    print(f"Inserting {len(bbf_contacts)} contacts using bulk API...")
    print("(Bulk API automatically batches in 200-record chunks)\n")

    try:
        results = bbf_sf.bulk.Contact.insert(bbf_contacts)

        successful_inserts = []
        failed_inserts = []

        for i, result in enumerate(results):
            if result["success"]:
                successful_inserts.append(
                    {
                        "es_id": bbf_contacts[i]["ES_Legacy_ID__c"],
                        "bbf_id": result["id"],
                        "name": f"{bbf_contacts[i].get('FirstName', '')} {bbf_contacts[i].get('LastName', '')}".strip(),
                        "bbf_account_id": bbf_contacts[i]["AccountId"],
                    }
                )
            else:
                failed_inserts.append(
                    {
                        "es_id": bbf_contacts[i]["ES_Legacy_ID__c"],
                        "name": f"{bbf_contacts[i].get('FirstName', '')} {bbf_contacts[i].get('LastName', '')}".strip(),
                        "errors": result["errors"],
                        "bbf_account_id": bbf_contacts[i]["AccountId"],
                    }
                )

        print(f"✅ Successfully inserted: {len(successful_inserts)} contacts")
        print(f"❌ Failed to insert: {len(failed_inserts)} contacts")

        if len(failed_inserts) > 0:
            print(f"\nFailed Contacts (first 5):")
            for item in failed_inserts[:5]:
                print(f"  - {item['name']} (ES ID: {item['es_id']})")
                print(f"    Errors: {item['errors']}")

        if len(successful_inserts) > 0:
            print(f"\nSample successful insert:")
            sample = successful_inserts[0]
            print(f"  ES Contact ID:  {sample['es_id']}")
            print(f"  BBF Contact ID: {sample['bbf_id']}")
            print(f"  Name:           {sample['name']}")
            print(f"  BBF Account ID: {sample['bbf_account_id']}")

    except Exception as e:
        print(f"❌ Error during bulk insert: {e}")
        successful_inserts = []
        failed_inserts = []


INSERTING CONTACTS TO BBF
Inserting 15665 contacts using bulk API...
(Bulk API automatically batches in 200-record chunks)

✅ Successfully inserted: 15665 contacts
❌ Failed to insert: 0 contacts

Sample successful insert:
  ES Contact ID:  0030B00001siX3WQAU
  BBF Contact ID: 003Ea00001LHPVGIA5
  Name:           Bill Brooks
  BBF Account ID: 001Ea00001LEf81IAD


In [7]:
# === UPDATE ES WITH BBF IDS ===

print("\n" + "=" * 80)
print("UPDATING ES WITH BBF IDS")
print("=" * 80)

if len(successful_inserts) == 0:
    print("⚠️  No contacts to update in ES")
    es_update_results = []
else:
    # Build update records for ES
    es_updates = [
        {"Id": item["es_id"], "BBF_New_Id__c": item["bbf_id"]}
        for item in successful_inserts
    ]

    print(f"Updating {len(es_updates)} Contacts in ES with BBF_New_Id__c...")

    try:
        es_update_results = es_sf.bulk.Contact.update(es_updates)

        success_count = sum(1 for r in es_update_results if r["success"])
        error_count = sum(1 for r in es_update_results if not r["success"])

        print(f"✅ Successfully updated: {success_count} Contacts in ES")
        print(f"❌ Failed to update: {error_count} Contacts in ES")

        if error_count > 0:
            print("\nFirst 10 update failures:")
            fail_count = 0
            for i, r in enumerate(es_update_results):
                if not r["success"] and fail_count < 10:
                    print(f"  - {es_updates[i]['Id']}: {r['errors']}")
                    fail_count += 1

    except Exception as e:
        print(f"❌ Error during ES update: {e}")
        es_update_results = []


UPDATING ES WITH BBF IDS
Updating 15665 Contacts in ES with BBF_New_Id__c...
✅ Successfully updated: 15665 Contacts in ES
❌ Failed to update: 0 Contacts in ES


In [8]:
# === CREATE EXCEL OUTPUT ===

print("\n" + "=" * 80)
print("CREATING EXCEL OUTPUT")
print("=" * 80)

# Create workbook
wb = Workbook()
ws1 = wb.active
ws1.title = "Migration Results"

# Styling
header_font = Font(bold=True, size=12, color="FFFFFF")
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
header_alignment = Alignment(horizontal="center", vertical="center")
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)

# Status colors
status_colors = {
    "SUCCESS": "C6EFCE",  # Green
    "FAILED": "FFC7CE",  # Red
    "SKIPPED": "FFEB9C",  # Yellow
}

# Build all results
all_results = []

for item in successful_inserts:
    all_results.append(
        {
            "ES_Contact_Id": item["es_id"],
            "BBF_Contact_Id": item["bbf_id"],
            "BBF_Account_Id": item["bbf_account_id"],
            "Contact_Name": item["name"],
            "Status": "SUCCESS",
            "Error": "",
        }
    )

for item in failed_inserts:
    all_results.append(
        {
            "ES_Contact_Id": item["es_id"],
            "BBF_Contact_Id": "",
            "BBF_Account_Id": item["bbf_account_id"],
            "Contact_Name": item["name"],
            "Status": "FAILED",
            "Error": str(item["errors"]),
        }
    )

for item in skipped_no_bbf_account:
    all_results.append(
        {
            "ES_Contact_Id": item["es_id"],
            "BBF_Contact_Id": "",
            "BBF_Account_Id": "",
            "Contact_Name": item["name"],
            "Status": "SKIPPED",
            "Error": f"No BBF Account ID (ES Account: {item['es_account_id']})",
        }
    )

# --- SHEET 1: Migration Results ---
headers1 = [
    "ES Contact ID",
    "BBF Contact ID",
    "BBF Account ID",
    "Contact Name",
    "Status",
    "Error",
]
ws1.append(headers1)

for col, header in enumerate(headers1, 1):
    cell = ws1.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment

for row_idx, r in enumerate(all_results, 2):
    ws1.append(
        [
            r["ES_Contact_Id"],
            r["BBF_Contact_Id"],
            r["BBF_Account_Id"],
            r["Contact_Name"],
            r["Status"],
            r["Error"],
        ]
    )
    fill_color = status_colors.get(r["Status"], "FFFFFF")
    for col in range(1, len(headers1) + 1):
        cell = ws1.cell(row=row_idx, column=col)
        cell.fill = PatternFill("solid", fgColor=fill_color)
        cell.border = thin_border

for col in ws1.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws1.column_dimensions[col[0].column_letter].width = min(max_length + 2, 60)

ws1.freeze_panes = "A2"

# --- SHEET 2: Summary ---
ws2 = wb.create_sheet("Summary")
ws2.append(["ES → BBF Contact Migration Summary"])
ws2["A1"].font = Font(bold=True, size=14)
ws2.append([])
ws2.append(["Run Type:", "TEST MODE" if TEST_MODE else "FULL MIGRATION"])
ws2.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws2.append(["Owner ID:", OWNER_ID])
ws2.append([])
ws2.append(["Metric", "Count"])
ws2["A7"].font = Font(bold=True)
ws2["B7"].font = Font(bold=True)
ws2.append(["Total Contacts Processed", len(all_results)])
ws2.append(["Successful Inserts", len(successful_inserts)])
ws2.append(["Failed Inserts", len(failed_inserts)])
ws2.append(["Skipped (No BBF Account)", len(skipped_no_bbf_account)])
ws2.append(
    [
        "Success Rate",
        (
            f"{len(successful_inserts)/len(all_results)*100:.1f}%"
            if len(all_results) > 0
            else "0%"
        ),
    ]
)

# --- SHEET 3: ID Mapping ---
ws3 = wb.create_sheet("ID Mapping")
headers3 = ["ES Contact ID", "BBF Contact ID", "BBF Account ID", "Contact Name"]
ws3.append(headers3)

for col, header in enumerate(headers3, 1):
    cell = ws3.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment

for item in successful_inserts:
    ws3.append([item["es_id"], item["bbf_id"], item["bbf_account_id"], item["name"]])

for col in ws3.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws3.column_dimensions[col[0].column_letter].width = min(max_length + 2, 50)

ws3.freeze_panes = "A2"

# --- SHEET 4: Failed Inserts ---
ws4 = wb.create_sheet("Failed Inserts")
headers4 = ["ES Contact ID", "Contact Name", "BBF Account ID", "Error Details"]
ws4.append(headers4)

for col, header in enumerate(headers4, 1):
    cell = ws4.cell(row=1, column=col)
    cell.font = Font(bold=True, size=12, color="FFFFFF")
    cell.fill = PatternFill(start_color="FF4444", end_color="FF4444", fill_type="solid")
    cell.alignment = header_alignment

for item in failed_inserts:
    ws4.append(
        [
            item["es_id"],
            item["name"],
            item["bbf_account_id"],
            str(item["errors"]),
        ]
    )

for col in ws4.columns:
    max_length = max(len(str(cell.value)) for cell in col) if list(col) else 10
    ws4.column_dimensions[col[0].column_letter].width = min(max_length + 2, 70)

ws4.freeze_panes = "A2"

# Save workbook
wb.save(output_file)
print(f"\n✅ Excel output saved to: {output_file}")
print(f"   📊 Sheet 1: Migration Results ({len(all_results)} contacts, color-coded)")
print(f"   📈 Sheet 2: Summary (metrics and stats)")
print(f"   🔗 Sheet 3: ID Mapping ({len(successful_inserts)} successful mappings)")
print(f"   ⚠️  Sheet 4: Failed Inserts ({len(failed_inserts)} failures)")


CREATING EXCEL OUTPUT

✅ Excel output saved to: es_bbf_contact_migration_20260123_114209.xlsx
   📊 Sheet 1: Migration Results (15665 contacts, color-coded)
   📈 Sheet 2: Summary (metrics and stats)
   🔗 Sheet 3: ID Mapping (15665 successful mappings)
   ⚠️  Sheet 4: Failed Inserts (0 failures)


In [9]:
# === FINAL SUMMARY ===

print("\n" + "=" * 80)
print("MIGRATION COMPLETE")
print("=" * 80)
print(f"ES Contacts queried: {len(es_contacts)}")
print(f"BBF Contacts inserted: {len(successful_inserts)}")
print(
    f"Success rate: {len(successful_inserts)/len(es_contacts)*100:.1f}%"
    if len(es_contacts) > 0
    else "N/A - No contacts processed"
)
print(f"\nExcel output: {output_file}")

if TEST_MODE:
    print("\n🔄 TEST MODE complete. Only migrated " + str(TEST_LIMIT) + " contacts.")
    print("   To migrate ALL contacts, set TEST_MODE = False in Cell 2 and re-run.")
else:
    print("\n✅ FULL MIGRATION complete!")
    print("   Contact migration finished.")
    print("   Next: Migrate BAN__c, BAN_Contact__c, Opportunities, etc.")

if len(failed_inserts) > 0:
    print(f"\n⚠️  {len(failed_inserts)} contacts failed to insert")
    print("   Check 'Failed Inserts' sheet in Excel for details")


MIGRATION COMPLETE
ES Contacts queried: 15665
BBF Contacts inserted: 15665
Success rate: 100.0%

Excel output: es_bbf_contact_migration_20260123_114209.xlsx

✅ FULL MIGRATION complete!
   Contact migration finished.
   Next: Migrate BAN__c, BAN_Contact__c, Opportunities, etc.


---
## Next Steps: Additional Object Migration

After Contact migration is complete, use the **ID Mapping sheet** from this Excel output to migrate related objects:

1. **BAN__c** (needs Account ID) - Billing Account Numbers
2. **BAN_Contact__c** (needs BAN + Contact IDs) - Junction object
3. **Opportunity** (needs Account + Contact IDs)
4. **Opportunity_Site__c** (needs Opportunity ID)

## Cleanup Apex (if needed)

### Delete Migrated Contacts from BBF
```apex
List<Contact> c = [SELECT Id, Name FROM Contact WHERE Owner.Name = 'Everstream Legacy'];
System.debug(c);
delete c;
```

### Remove BBF_New_Id__c from ES Contacts
```apex
List<Contact> contactIds = [SELECT Id, BBF_New_Id__c FROM Contact WHERE BBF_New_Id__c != NULL];
System.debug(contactIds);
for (Contact c : contactIds){
    c.BBF_New_Id__c = NULL;
}
update contactIds;
```

In [10]:
# Install required packages (run if needed)
# !pip install simple-salesforce pandas openpyxl